In [1]:
import time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras

from qiskit import BasicAer
from qiskit.circuit.library import ZZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua import QuantumInstance
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit_machine_learning.datasets import ad_hoc_data
from qiskit_machine_learning.algorithms import QSVC

Loading Datasets

In [2]:
# Direct Kernel Set Training Data
dks_train_1 = pd.read_csv('data/Direct_Kernel_Set_I_Training.csv')

# Direct Kernel Set Classifcation Data
dks_class_1 = pd.read_csv('data/Direct_Kernel_Set_I_Classifications.csv')

x1_train = dks_class_1.iloc[:, 1:3]
y1_train = dks_class_1.iloc[:, 3]

x1_test = dks_train_1.iloc[:, 1:3]
y1_test = dks_train_1.iloc[:, 3]

In [3]:
x2_train, y2_train, x2_test, y2_test = ad_hoc_data(training_size = 80, test_size = 20, n=2, gap = 0.2, one_hot = False)

In [10]:
(x3_train, y3_train), (x3_test, y3_test) = keras.datasets.mnist.load_data()
x3_train = x3_train.reshape((60000, 28*28))
x3_test = x3_test.reshape((-1, 28*28))
x3_train.shape
num_dims = 15
idx = list(range(0, 28*28, int(28*28 / num_dims) + 1))
x3_train[:10, idx]

array([[  0,   0,   0,  26, 253, 205,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0, 237, 252, 253, 253,  57,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 253,   0,   0,   0,   0,
         96,   0],
       [  0,   0,   0, 253,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0, 253, 190, 252, 232,   0,   0,   0,   0,   0,
         42,   0],
       [  0,   0,   0,   7, 224, 252, 128,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 253,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0, 253, 252,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 254,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0, 253,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=uint8)

Training, Timing, and Evaluating Datasets

In [5]:
# First dataset (from paper)
random_seed = 10598
feature_map = ZZFeatureMap(feature_dimension=2,
                                 reps=1, entanglement='full')
backend = QuantumInstance(BasicAer.get_backend('statevector_simulator'), shots=1,
                                seed_simulator=random_seed, seed_transpiler=random_seed)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

qsvc = QSVC(quantum_kernel=kernel)
start_time = time.time()
qsvc.fit(x1_train, y1_train)#train
train_time = time.time() - start_time
qsvc_score = qsvc.score(x1_test, y1_test) #predict
test_time = time.time() - start_time - train_time
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(qsvc_score)

Training time:1.5510003566741943[sec]
Test time:0.7499992847442627[sec]
0.9


In [6]:
# Second dataset (ad hoc data)
random_seed = 10598
feature_map = ZZFeatureMap(feature_dimension=2,
                                 reps=1, entanglement='full')
backend = QuantumInstance(BasicAer.get_backend('statevector_simulator'), shots=1,
                                seed_simulator=random_seed, seed_transpiler=random_seed)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

qsvc = QSVC(quantum_kernel=kernel)
start_time = time.time()
qsvc.fit(x2_train, y2_train)#train
train_time = time.time() - start_time
qsvc_score = qsvc.score(x2_test, y2_test) #predict
test_time = time.time() - start_time - train_time
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(qsvc_score)

Training time:0.4049994945526123[sec]
Test time:0.27468252182006836[sec]
0.9


In [11]:
# Third dataset (mnist dataset)
random_seed = 10598
feature_map = ZZFeatureMap(feature_dimension=num_dims,
                                 reps=1, entanglement='full')
backend = QuantumInstance(BasicAer.get_backend('statevector_simulator'), shots=1,
                                seed_simulator=random_seed, seed_transpiler=random_seed)
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=backend)

qsvc = QSVC(quantum_kernel=kernel)
start_time = time.time()
qsvc.fit(x3_train[:10, idx], y3_train[:10])#train
train_time = time.time() - start_time
qsvc_score = qsvc.score(x3_test[:10, idx], y3_test[:10]) #predict
test_time = time.time() - start_time - train_time
print ("Training time:{0}".format(train_time) + "[sec]")
print ("Test time:{0}".format(test_time) + "[sec]")

print(qsvc_score)

Training time:2.281454563140869[sec]
Test time:3.9150776863098145[sec]
0.2
